In [1]:
import pandas as pd
names = pd.read_csv('names.csv')

In [2]:
data = pd.read_json('/content/drive/MyDrive/Hammasir-Final/doctors.json')

In [ ]:
no_name = data[data["name"].isna()]
s_name = no_name["title"].str.split(" ",n=1)
f_name = s_name.str[0]
no_name["name"] = f_name

In [4]:
data.loc[no_name.index, 'name'] = no_name['name']

In [ ]:
names["Gender"] = names["Gender"].replace("پسر", "M")
names["Gender"] = names["Gender"].replace("دختر", "F")
names.head()

In [ ]:
import re
def find_gender(data_name):
    # Escape special characters in cyka_name to treat it as a literal string
    escaped_name = re.escape(data_name)
    match = names[names['Name'].str.contains(escaped_name, regex=False)]
    if not match.empty:
        return match['Gender'].iloc[0]
    else:
        return None

data['gender'] = data['name'].apply(find_gender)
data.head()

In [7]:
cent = data["centers"]
matab = []
for c in cent:
  temp_list = []
  for item in c:
    temp = {}
    temp["city"] = item["city_name"]
    temp["number"] = item["display_number"]
    temp["address"] = item["address"]
    temp["province_name"] = item["province_name"]
    temp_list.append(temp)
  matab.append(temp_list)
data["matab"] = matab

In [ ]:
base_dataset = data[["title", "display_expertise", "gender", "star",
                     "rates_count", "number_of_visits", "view",
                     "insurances", "experience", "rate_info", "medical_code", "matab"]]
base_dataset.head()

In [ ]:
base_dataset = base_dataset.drop_duplicates(subset=["title", "medical_code"], keep="first")

In [26]:
base_dataset.to_json("base_dataset.json")
base_dataset.to_csv("base_dataset.csv")